In [1]:
import numpy as np

In [44]:
from random import randint

class Graph:
    """
    Graph data structure.
    """

    def __init__(self, fname = None, numVertices = None, numEdges = None, weightRange = None, directed = True, empty = False):
        """
        Generates a weighted graph.
        """
        self.adjacent = {}
        self.weight = {}
        if empty:
            self.emptyGraph(numVertices)
        elif fname == None:
            if any(arg == None for arg in (numVertices, numEdges, weightRange)):
                numVertices, numEdges, weightRange = map(int, input("numVertices, numEdges, weightRange: ").split())

            self.randomGraph(numVertices, numEdges, weightRange, directed)

        else:
            self.loadGraph(fname, directed)


    def numVertices(self):
        """
        Returns the number of vertices in the graph.
        """
        return len(self.adjacent)


    def vertices(self):
        """
        Returns the list of vertices in the graph.
        """
        return range(self.numVertices())


    def edges(self):
        """
        Returns a generator containing the edges in the graph.
        """
        return ((fromVertex,toVertex) for fromVertex in self.vertices() for toVertex in self.adjacent[fromVertex])


    def addDirectedEdge(self, fromVertex, toVertex, weight):
        """
        Inserts a weighted directed edge into the graph.
        """
        self.adjacent.setdefault(fromVertex, set()).add(toVertex)
        self.weight[(fromVertex, toVertex)] = weight


    def addUndirectedEdge(self, fromVertex, toVertex, weight):
        """
        Inserts a weighted undirected edge into the graph.
        """
        self.addDirectedEdge(fromVertex, toVertex, weight)
        self.addDirectedEdge(toVertex, fromVertex, weight)


    def randomGraph(self, numVertices, numEdges, weightRange, directed):
        """
        Generates a random graph.
        """
        addEdge = self.addDirectedEdge if directed else self.addUndirectedEdge

        for vertex in range(numVertices):
            self.adjacent[vertex] = set()

        for edge in range(numEdges):
            fromVertex = toVertex = None
            while fromVertex == toVertex:
                fromVertex = randint(0, numVertices-1)
                toVertex   = randint(0, numVertices-1)

            weight = randint(0, weightRange)
            addEdge(fromVertex, toVertex, weight)

    def emptyGraph(self, numVertices):
        """
        Generates a random graph.
        """
        addEdge = self.addDirectedEdge

        for vertex in range(numVertices):
            self.adjacent[vertex] = set()

            



    def adjacentStr(self, fromVertex):
        """
        Returns a string representing the neighborhood of the
        given vertex.
        """
        return ", ".join(f"({toVertex}, {self.weight[(fromVertex, toVertex)]})" for toVertex in self.adjacent[fromVertex])


    def __str__(self):
        """
        Returns a string representing the graph.
        """
        return "\n".join(f"{vertex}: {self.adjacentStr(vertex)}" for vertex in range(self.numVertices()))


    def __repr__(self):
        """
        Represents the graph.
        """
        return str(self)


In [45]:
def APSP_DynamicProgramming(graph):
    """
    Applies dynamic programming to compute all-pairs
    shortest paths.
    """
    n = graph.numVertices()
    dst = [[float('inf') for v in range(n)] for u in range(n)]
    nxt = [[None for v in range(n)] for u in range(n)]

    for u in range(n):
        dst[u][u] = 0
        for v in graph.adjacent[u]:
            dst[u][v] = graph.weight[(u, v)]
            nxt[u][v] = v

    for m in range(1, n):
        for u in graph.vertices():
            for (x, v) in graph.edges():
                new_dst_uv = dst[u][x] + graph.weight[(x, v)]
                if new_dst_uv < dst[u][v]:
                    dst[u][v] = new_dst_uv
                    nxt[u][v] = nxt[u][x]


    # Check for negative weight cycles
    for u in graph.vertices():
        if any(dst[u][v] > dst[u][x] + graph.weight[(x,v)] for (x, v) in graph.edges()):
            return (None, None)

    return (dst, nxt)

In [55]:
def showResults(graph, dst, pointer):
    """
    Shows all-pairs shortest paths information.
    """
    if dst == None:
        print(None)
        return

    print("Distances:")
    for (v, row) in zip(graph.vertices(), dst):
        rowi = "\t".join(str(i) for i in row)
        print(f"{v}: {rowi}")

    #print("\nPath Pointers:")
    #for (v, row) in zip(graph.vertices(), pointer):
    #    rowi = "\t".join(str(i) for i in row)
    #   print(f"{v}: {rowi}")

In [56]:
graph = Graph(weightRange=0, numEdges=0,numVertices=8,empty=True)

graph.addDirectedEdge(1,2,6)
graph.addDirectedEdge(1,4,7)
graph.addUndirectedEdge(1,5,9)
graph.addDirectedEdge(2,6,5)
graph.addDirectedEdge(2,3,5)
graph.addDirectedEdge(3,7,3)
graph.addDirectedEdge(3,4,2)
graph.addDirectedEdge(4,7,1)
graph.addDirectedEdge(5,6,4)
graph.addDirectedEdge(5,7,8)
graph.addDirectedEdge(6,7,4)


print(str(graph))

print("\n\nDynamic Programming:")
print("-"*25)
dst, nxt = APSP_DynamicProgramming(graph)
showResults(graph, dst, nxt)

0: 
1: (2, 6), (4, 7), (5, 9)
2: (3, 5), (6, 5)
3: (4, 2), (7, 3)
4: (7, 1)
5: (1, 9), (6, 4), (7, 8)
6: (7, 4)
7: 


Dynamic Programming:
-------------------------
Distances:
0: 0	inf	inf	inf	inf	inf	inf	inf
1: inf	0	6	11	7	9	11	8
2: inf	inf	0	5	7	inf	5	8
3: inf	inf	inf	0	2	inf	inf	3
4: inf	inf	inf	inf	0	inf	inf	1
5: inf	9	15	20	16	0	4	8
6: inf	inf	inf	inf	inf	inf	0	4
7: inf	inf	inf	inf	inf	inf	inf	0
